In [14]:
# ML
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform, pdist

# Data handling
import pandas as pd
import numpy as np

from collections import Counter
import operator

# Plotting
%matplotlib inline
# import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import seaborn as sns

# Read and write
from pathlib import Path # filesystem

## sklearn is giving me
## sklearn/model_selection/_search.py:814: DeprecationWarning: 
## The default of the `iid` parameter will change from True to False in version 0.22 
## and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
##  DeprecationWarning)

# Suppress that
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Initial setup

## GLOBAL VARIABLES

These are variables that are used throughout the notebook

In [15]:
# Set this to False if you don't want to scale the feature values
SCALE_DATA = True

In [16]:
# Only change these if you know what you are doing

# A dictionary that maps original feature names to final ones
FEATURES_MAP = dict(zip(['jaccard_score', 'same_score', 
                            'inwards_score', 'outwards_score', 
                            'avg_distance', 'mean_ani', 
                            'mean_aai'],
                        ['Co-occurrence', 'Co-orientation',
                            'Convergent', 'Divergent',
                            'Average Distance', 'Mean ANI',
                            'Mean AAI']
                       )
                   )

# A list to only get the features
# Useful for slicing data frames
FEATURES = ['jaccard_score', 'same_score', 
            'inwards_score', 'outwards_score', 
            'avg_distance', 'mean_ani', 
            'mean_aai']
                    
# Features with label appended
# useful for plotting based on label
FEAT_LAB = FEATURES.copy()
FEAT_LAB.append('label')

## HELPER FUNCTIONS

* Reading data

Some of these functions were written **before** filtering for max distance, so they try to address this issue.

In [17]:
def read_scores_table(scores_fp, label=None):
    """
    Read a table from a tsv file provided as a path.
    Return a dataframe.
    
    If label is set, append a column named `label` with
    the provided value
    """
    scores_df = pd.read_csv(scores_fp, sep='\t')
    scores_df['interaction'] = scores_df['pvog1'] + '-' + scores_df['pvog2']
    # Unpack features list because I want interaction prepended.
    scores_df = scores_df[['interaction', *FEATURES]]    
    if label is not None:
        scores_df['label'] = label
    return scores_df


def concat_data_frames(pos_df, 
                       neg_df, 
                       subsample=False,
                       clean=True,
                       is_scaled=True,
                      ):
    """
    Concatenate two dataframes.
    
    subsample:bool 
        Subsample the `neg_df` to a number of
        observations equal to the number of pos_df 
        (balance datasets)
    
    clean:bool
        Remove observations with a value of `avg_distance` == 100000 or 1.
    
    is_scaled:bool
        The features have been scaled to a range 0-1
    
    Return:
    concat_df: pd.DataFrame
        The concatenated data frame
    """
    
    if clean is True:
        pos_df = remove_ambiguous(pos_df)
        neg_df = remove_ambiguous(neg_df)
    
    n_positives = pos_df.shape[0]
    n_negatives = neg_df.shape[0]
    
    # Remove possible duplicate interactions from the negatives
    # This might happen because of the random selection when creating the set
    # Why I also select more negatives to begin with
    neg_df = neg_df.loc[~neg_df['interaction'].isin(pos_df['interaction'])]
    
    if (n_positives != n_negatives) and (subsample is True):
        neg_df = neg_df.sample(n=n_positives, random_state=1)
    concat_df = pd.concat([pos_df, neg_df])
    
    assert concat_df[concat_df.duplicated(subset=['interaction'])].empty == True, concat_df.loc[concat_df.duplicated(subset=['interaction'], keep=False)]
    
    return concat_df

def scale_df(input_df):
    """
    Scale all feature values in the data frame to [0-1].
    """
    maxes = input_df[FEATURES].max(axis=0)
    scaled_data = input_df[FEATURES].divide(maxes)
    if 'label' in input_df.columns:
        scaled_df = pd.concat([input_df['interaction'], scaled_data, input_df['label']], axis=1)
    else:
        scaled_df = pd.concat([input_df['interaction'], scaled_data], axis=1)
    return scaled_df

def remove_ambiguous(input_df):
    """
    Select observations in the `input_df` that have feature values
    """
    df_clean = input_df[(input_df.jaccard_score != 0)] # This is true if they don't co-occur
    return df_clean


# Analysis

From this point on all the magic happens

In [18]:
# Get all the files in a list of Path objects
filepaths = list(map(Path, snakemake.input))
# Remove the last element
filtered_scores_tsv = filepaths.pop(-1)
negatives_fp_list = []
for fp in filepaths:
    # Grab the positives
    if 'positives' in fp.name:
        positives_tsv = fp
    else:
        # Append to the list of negatives
        negatives_fp_list.append(fp)

In [19]:
# Positives are always the same
pos_df = read_scores_table(positives_tsv, label = 1)

# Negatives are stored in the list above
# Select one for visualization
neg_df = read_scores_table(negatives_fp_list[3], label = 0)
if SCALE_DATA is True:
    pos_df = scale_df(pos_df)
    neg_df = scale_df(neg_df)
    
posneg = concat_data_frames(pos_df, neg_df,
                           clean=True,
                           subsample=True,
                           is_scaled=SCALE_DATA)

In [20]:
def plot_features_correlation(data_df):
    # Subset to features only
    cor_data = data_df[FEATURES]
    cor_df = cor_data.corr()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cor_df, annot=True, 
            cmap=sns.color_palette("RdBu_r"), 
            vmin=-1, 
            vmax=1, 
            cbar_kws = {"shrink" : 0.5, 
                        "ticks" : [-1, -0.5, 0, 0.5, 1],
                       }
           )    
    plt.show()

## THIS IS NO LONGER USED in favor of the pairplot
# you can call it with
#  plot_features_dist(posneg, FEATURES, is_scaled = SCALE_DATA)
def plot_features_dist(data, features, is_scaled=False):
    """
    Make a gridplot of all features distributions
    
    data: A pd.DataFrame with features and a label column
    features: A list of features to use
    """
    if is_scaled:
        distance_threshold = 1
    else:
        distance_threshold = 1000000
        
    df_list=[data.loc[:,[f, 'label']] for f in features]
    # Initialize a figure
    fig, axes = plt.subplots(3, 3, figsize = (8,4), dpi=300)
    # Flatten the axes for plotting in the correct ax object
    ax_list = axes.flatten()

    # Workaround to add the legend
    red_patch = mpatches.Patch(color='red', label='positives', alpha=0.5)
    blue_patch = mpatches.Patch(color='b', label='negatives', alpha=0.5)
    plt.figlegend(handles=[red_patch, blue_patch], loc='lower center')
    # Make the plots
    for i in range(len(df_list)):
        df=df_list[i]
        if features[i] == "avg_distance":
            x0 = df.loc[((df.label==0) & (df.avg_distance != distance_threshold)), features[i]]
            x1 = df.loc[((df.label==1) & (df.avg_distance != distance_threshold)), features[i]]
        else:
            x0 = df.loc[df.label==0, features[i]] 
            x1 = df.loc[df.label==1, features[i]]
#     a1 = sns.distplot(x0, ax=axes[coords[0], coords[1]], axlabel=features[i],color='b', label='negatives')
#     a2 =sns.distplot(x1, ax=axes[coords[0], coords[1]], axlabel=features[i], color='r', label='positives')
        ax = ax_list[i]
        ax.hist(x0, color = 'b', alpha=.5)
        ax.hist(x1, color = 'r', alpha= .5)
        ax.set_title("{} (N={}/{})".format(features[i], 
                                           x0.shape[0], 
                                           x1.shape[0]), 
                                           fontsize=8
                    )

    plt.subplots_adjust(hspace=0.7)

    axes[2,2].remove()
    axes[2,1].remove()    

In [21]:
pair = sns.pairplot(posneg[FEAT_LAB],
                    hue = 'label', 
                    vars=FEATURES,
                    markers = ["+", "x"],
                    palette = {
                        0 : 'b',
                        1 : 'r',
                    },
                    corner=True,
#              diag_kind="hist",
                    plot_kws = {
                        "alpha": 0.65,
                    },
                    diag_kws={
                        "clip" : [0. , 1.],
                    },
                )
pair.set(xlim=(-0.1, 1.1))

In [22]:
plot_features_correlation(posneg)

## Cluster interactions

In [23]:
# Make a copy of the original df for plotting
d_posneg = posneg

D_posneg = pdist(d_posneg[FEATURES])
D_posneg = D_posneg / D_posneg.max() # Scale the distances to be [0-1]
Z_posneg = linkage(D_posneg, method="average")

In [24]:
# Append a color column based on the label value
# https://stackoverflow.com/a/26887820

def color_label_interaction(row):
    if row['label'] == 1:
        return 'red'
    else:
        return 'blue'

d_posneg['color'] = d_posneg.apply(lambda row: color_label_interaction(row), axis=1)

In [25]:
plt.figure(figsize=(20, 30))
dn = dendrogram(Z_posneg,
                labels=d_posneg['interaction'].values, 
                orientation='left',
                leaf_font_size=12,
                color_threshold=0.1,
                above_threshold_color='black',
               )
plt.axvline(x=0.1, linestyle='--')
plt.title("Average linkage based on Euclidean distances", size=14)
# plt.tick_params(axis='y', labelsize=12, direction='out')
ax=plt.gca()
ylbls = ax.get_ymajorticklabels()
for lbl in ylbls:
    lbl.set_color(d_posneg.loc[d_posneg.interaction == lbl.get_text(), 'color'].values[0])
plt.show()


### Distribution of distances pos vs neg

In [26]:
D_pos = pdist(pos_df[FEATURES])
D_neg = pdist(neg_df[FEATURES])

In [27]:
fig, ax = plt.subplots(figsize=(10,8))

ax = sns.kdeplot(D_pos,
                 fill=True,
                 alpha = 0.5,
                 color='r',
                 
                )
ax = sns.kdeplot(D_neg,
                 fill=True,
                 alpha = 0.5,
                 color='b'
                )

ax.set_title("Distances distribution", size=14)
ax.set_xlabel("Distance", size=14)
ax.set_ylabel("Density", size=14)
ax.legend(["Positives", "Negatives"])
ax.tick_params(axis='both', which='major', labelsize=12)

# Random Forest

## Model selection and hyperparameter tuning

Define a search space for the parameters to be sampled from. 
Less exhaustive than a full grid search, but quicker.

In [28]:
# Number of trees
n_estimators = [int(x) for x in range(100,5000,200)]

# Number of features to consider at every split
# 'auto' uses sqrt(n_features), a float uses int(float * n_features)
max_features = ['auto', .5, 1.]

# Maximum number of levels in each tree
max_depth = [int(x) for x in range(10, 60, 10)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method for selecting samples for training
bootstrap = [True, False]

# Put them all in a dictionary
params_space = {
    "max_features": max_features,
    "n_estimators" : n_estimators,
    "max_depth" : max_depth,
    "min_samples_split" : min_samples_split,
    "min_samples_leaf" : min_samples_leaf,
    "bootstrap": bootstrap,
    }

In [39]:
# Create a color mapping for the negative sets

negative_sets = []
for negative in negatives_fp_list:
    set_name = negative.name.split('.')[0]
    negative_sets.append(set_name)
    
color_palette = sns.color_palette("colorblind", len(negative_sets))

color_dict = dict(zip(negative_sets, color_palette))

In [30]:
def calculate_fdr(confusion_matrix):
    """
    Calculate FDR based on the values of the confusion_matrix
    
    Args:
        confusion_matrix: 2d np.array: The confusion matrix returned
            by sklearn.metrics.confusion_matrix
    Return:
        fdr: float: A single value representing the FDR of the model
            based on count values of FP/FP+TP
    """
    tn, fp, fn, tp = confusion_matrix.ravel()
    fdr = fp / (fp + tp)
    return round(fdr, 3)
    

def get_metric_value(metric_string, 
                     truth_array, 
                     prediction_array, 
                     prediction_proba_array):
    """
    Helper function to get metric values based on a string representation.
    """
    if metric_string == 'accuracy':
        return metrics.accuracy_score(truth_array, prediction_array)
    if metric_string == 'accuracy_abs':
        return metrics.accuracy_score(truth_array, prediction_array, normalize = False)
    if metric_string == 'precision':
        return metrics.precision_score(truth_array, prediction_array)
    if metric_string == 'recall':
        return metrics.recall_score(truth_array, prediction_array)
    if metric_string == 'f1_score':
        return metrics.f1_score(truth_array, prediction_array)
    if metric_string == 'roc_auc_score':
        return metrics.roc_auc_score(truth_array, prediction_proba_array[:,1])
    if metric_string == 'fpr':
        return metrics.roc_curve(truth_array, prediction_proba_array[:,1])[0]
    if metric_string == 'tpr':
        return metrics.roc_curve(truth_array, prediction_proba_array[:,1])[1]
    if metric_string == 'confusion_matrix':
        return metrics.confusion_matrix(truth_array, prediction_array)
    if metric_string == 'fdr':
        cf = metrics.confusion_matrix(truth_array, prediction_array)
        fdr = calculate_fdr(cf)
        return fdr
    if metric_string == 'mcc':
        return metrics.matthews_corrcoef(truth_array, prediction_array)

In [31]:
# A list of all the metrics we are storing
# _metrics so we don't override the imported sklearn.metrics
_metrics = [
    'accuracy', 
    'accuracy_abs', 
    'f1_score', 
    'precision', 
    'recall', 
    'roc_auc_score', 
    'fpr', 
    'tpr',
    'confusion_matrix',
    'fdr',
    'mcc'
    ]

# A dictionary that will hold all the values
# for the metrics above
all_metrics = {}
# A dictionary that holds the parameters for the best model
# after each iteration
best_models = {}

# A dictionary that holds feature importances
# for each iteration
feat_importances = {}

for i, negative in enumerate(negatives_fp_list):
    
    # Make a copy of the list with the negative_files
    copy_of_list = negatives_fp_list.copy()    
    # Get the name of the current negative
    training_set = negative.name.split('.')[0]
    # Read the current negative data in a data frame
    neg_df = read_scores_table(negative, label=0)
    # ... scale it
    neg = scale_df(neg_df)
    # Concatenate with the positive
    # A ground truth set is constructed
    posneg = concat_data_frames(pos_df, 
                                neg, 
                                subsample=True, 
                                clean=True, 
                                is_scaled=True)
    
    # Split the frame to features and the label
    X = posneg[FEATURES]
    y = posneg['label']
    
    # Train test split
    X_train, X_holdout, y_train, y_holdout = train_test_split(X,
                                                              y,
                                                              test_size=0.3,
                                                              random_state=1)
    
    # Instantiate the Randomized Search
    models = RandomizedSearchCV(
        estimator = RandomForestClassifier(random_state=1),
        param_distributions = params_space,
        n_iter = 500, # number of parameter settings to sample
        cv = 5, # Use 5-folds for CV
        random_state = 1, # Set random state for reproducibility
        n_jobs = snakemake.threads,
        verbose=1 # Print some messages to keep track of progress
        )
    # Execute the search
    search = models.fit(X_train, y_train)
    
    # Make a classifier out of the best model
    best_model = RandomForestClassifier(**search.best_params_, 
                                        random_state=1)
    
    # Fit the best model on the training data
    best_model.fit(X_train, y_train)
    
    # Predict labels and probabilities for the test/holdout set
    holdout_pred = best_model.predict(X_holdout)
    holdout_proba = best_model.predict_proba(X_holdout)
    
    ###############
    # Store results
    ###############
    # Create a tuple of the strings of the training set
    # as a key for the all_metrics dict
    self_key = (training_set, training_set)
    
    # The get_metric_value is defined above
    all_metrics[self_key] = {metric : get_metric_value(metric,
                                                       y_holdout,
                                                       holdout_pred,
                                                       holdout_proba)
                            for metric in _metrics}
    
    best_models[training_set] = best_model
    
    importances = best_model.feature_importances_      
    feat_importances[self_key] = dict(zip(FEATURES, importances))
    
    # Remove the current negative set file from the copy of the list
    # The rest will be used for validation
    validation_sets = [i for i in copy_of_list if i != negative]

    for validation_set in validation_sets:
        # Repeat the same process as above, but without optimizing
        validation_set_name = validation_set.name.split('.')[0]
        negg = read_scores_table(validation_set, label=0)
        scaled_negg = scale_df(negg)
        
        pn = concat_data_frames(pos_df, 
                                scaled_negg, 
                                subsample=True, 
                                clean=True, 
                                is_scaled=True)
        XX = pn[FEATURES]
        yy = pn['label']
        
        XX_train, XX_holdout, yy_train, yy_holdout = train_test_split(XX, yy, test_size=0.3, random_state=1)
        best_model.fit(XX_train, yy_train)
        holdout_pred = best_model.predict(XX_holdout)
        holdout_proba = best_model.predict_proba(XX_holdout)
        
        
        combo_key = (training_set, validation_set_name)
        all_metrics[combo_key] = {metric : get_metric_value(metric, yy_holdout, holdout_pred, holdout_proba) 
                                 for metric in _metrics}
        
        importances = best_model.feature_importances_      
        feat_importances[combo_key] = dict(zip(FEATURES, importances))
    
    print("Finished set : {}".format(i))

## Save results

In [32]:
# Results are stored in the results/RF directory
# Create it, or do nothing
RF_dir = Path("results/RF")
RF_dir.mkdir(exist_ok=True)

In [33]:
## metrics
metrics_df = pd.DataFrame.from_dict(all_metrics, orient='index', columns=_metrics)

# Append columns TS (Training Set) and VS (Validation Set)
# For slicing
metrics_df['TS'] = [i[0] for i in metrics_df.index.values]
metrics_df['VS'] = [i[1] for i in metrics_df.index.values]

# Pickle is used to store the fpr, tpr, confusion matrix values as arrays
# otherwise it messes up the column formatting
metrics_fp = RF_dir / Path('metrics.pkl')
metrics_tsv = RF_dir / Path('metrics.tsv')
metrics_df.to_pickle(metrics_fp)
## Drop the fpr,tpr, confusion_matrix (arrays) to write to tsv
metrics_df.drop(columns=['fpr', 
                         'tpr',
                         'confusion_matrix']).to_csv(metrics_tsv, 
                                        sep='\t', 
                                        index=False)

## features
features_df = pd.DataFrame.from_dict(feat_importances,
                                     orient = 'index',
                                     columns = FEATURES)
features_df['TS'] = [i[0] for i in features_df.index.values]
features_df['VS'] = [i[1] for i in features_df.index.values]
features_fp = RF_dir / Path("features.tsv")

features_df.to_csv(features_fp, sep='\t', index=False)


## models
models_dir = RF_dir / Path("models")
models_dir.mkdir(exist_ok=True)
for dataset in best_models:
    pkl_fname = Path('{}.RF.pkl'.format(dataset))
    pkl_fpath = models_dir.joinpath(pkl_fname)
    with open(pkl_fpath, 'wb') as fout:
        pickle.dump(best_models[dataset], fout)


## Best model across the board

In [34]:
# Calculate min, max, mean, std for all metrics across datasets
## this probably is not the most pythonic way of doing this

metrics_stats = {}
for i, negative in enumerate(negatives_fp_list):
    dataset_name = negative.name.split('.')[0]
    # Get the stats for a particular dataset and trnaspose it
    set_df = metrics_df.loc[metrics_df["TS"] == dataset_name, _metrics].describe().T
    
    metrics_stats[dataset_name] = {'accuracy_mean' : set_df.loc['accuracy', 'mean'],
                                 'accuracy_std' : set_df.loc['accuracy', 'std'],
                                'accuracy_min' : set_df.loc['accuracy', 'min'],
                                'accuracy_max' : set_df.loc['accuracy', 'max'],
                                 'precision_mean' : set_df.loc['precision', 'mean'],
                                 'precision_std' : set_df.loc['precision', 'std'],
                                'precision_min' : set_df.loc['precision', 'min'],
                                'precision_max' : set_df.loc['precision', 'max'],
                                 'recall_mean' : set_df.loc['recall', 'mean'],
                                 'recall_std' : set_df.loc['recall', 'std'],
                                'recall_min' : set_df.loc['recall', 'min'],
                                'recall_max' : set_df.loc['recall', 'max'],
                                 'f1_score_mean': set_df.loc['f1_score', 'mean'],
                                 'f1_score_std': set_df.loc['f1_score', 'std'],
                                'f1_score_min' : set_df.loc['f1_score', 'min'],
                                'f1_score_max' : set_df.loc['f1_score', 'max'],
                                'roc_auc_score_mean': set_df.loc['roc_auc_score', 'mean'],
                                 'roc_auc_score_std': set_df.loc['roc_auc_score', 'std'],
                                'roc_auc_score_min' : set_df.loc['roc_auc_score', 'min'],
                                'roc_auc_score_max' : set_df.loc['roc_auc_score', 'max'],
                                'accuracy_abs_mean' : set_df.loc['accuracy_abs', 'mean'],
                                'accuracy_abs_std' : set_df.loc['accuracy_abs', 'std'],
                                'accuracy_abs_min' : set_df.loc['accuracy_abs', 'min'],
                                'accuracy_abs_max' : set_df.loc['accuracy_abs', 'max']
                               }

metrics_stats_df = pd.DataFrame.from_dict(metrics_stats, orient='index')

# Save them
metrics_stats_fp = RF_dir / Path("metrics.stats.tsv")
metrics_stats_df.to_csv(metrics_stats_fp, sep='\t',)


In [35]:
# Select the best model automatically

# Initialize a dictionary {N1 : 0, N2 : 0, ...}
dataset_scores = dict(zip(metrics_stats_df.T.columns, 
                          [0 for i in range(len(metrics_stats_df.T.columns))]))
# Iterate over the rows of the dataframe
for stat, row in metrics_stats_df.T.iterrows():
    if stat.endswith('std'):
        #row is a series. idxmax() returns the value of the index of the series
        dataset_scores[row.idxmin()] += 1 
    else:
        dataset_scores[row.idxmax()] += 1
        
# Select best scoring model
best_model = max(dataset_scores.items(), key=operator.itemgetter(1))[0]
print("AND THE BEST MODEL IS...: {}".format(best_model))

best_model_fp = RF_dir / Path("best_model.pkl")
# Should be identical with the one in the models dir
with open(best_model_fp, 'wb') as fout:
    pickle.dump(best_models[best_model], fout)

# Write the best model id in a file.
## This is used internally in the workflow as a checkpoint
# Should be a file containing just the id , e.g. N2
best_model_id_fp = RF_dir / Path("best_model_id.txt")
with open(best_model_id_fp, 'w') as fout:
    fout.write(best_model)


# PLOTS

## Metrics

In [36]:
def plot_dataset_roc_curve(training_set,
                           data_df,
                           plot_all=True,
                           ax=None,
                           label_xy=True):
    """
    Plot the roc curve for a dataset.
    training_set:str
        The name of the dataset
    data_df: pandas.Dataframe
        A data frame that holds all metrics
    plot_all:bool
        If roc curves for all other datasets used for validation
        should be plotted
    ax:pyplot.axes
        An axes object (used for subplotting)
    label_xy:bool
        If axes x,y should be annotated
    
    """
    
    if ax is None:
        ax = plt.gca()
        
    rc = ax.plot([0,1], [0,1], linestyle='--', color='k', label='Baseline')
    
    # Select all rows for a particular dataset used for the optimization
    # and create a new dataframe
    train_df = data_df.loc[data_df['TS'] == training_set]
    # Grab the fpr and tpr arrays for that set
    
    fpr = train_df.loc[train_df['VS'] == training_set, 'fpr'].values
    tpr = train_df.loc[train_df['VS'] == training_set, 'tpr'].values
    # Get the auc_score value
    auc_score = data_df.loc[(training_set, training_set), 'roc_auc_score']
    # Plot the roc curve
    ## fpr and tpr are returned as an array of arrays
    ## with length == 1. Hence, we grab the first ellement which
    ## is the array itself
    rc = ax.plot(fpr[0], tpr[0], 
                 label="{}, auc={:.2f}".format(training_set, auc_score),
                 color = color_dict.get(training_set), 
                 linewidth=3, 
                 alpha=1)
    # For plotting all other datasets roc curves
    if plot_all is True:
        for validation_set in train_df['VS'].values:
            if validation_set == training_set:
                pass
            else:
                # This is done on the subset train_df
                fpr = train_df.loc[train_df['VS'] == validation_set, 'fpr'].values
                tpr = train_df.loc[train_df['VS'] == validation_set, 'tpr'].values
                auc_score = data_df.loc[(training_set, validation_set), 'roc_auc_score']
                rc = ax.plot(fpr[0], tpr[0], 
                             label="{}, auc={:.2f}".format(validation_set, auc_score),
                            color = color_dict.get(validation_set),
                             linewidth=1,
                            alpha=0.7)
            
    rc = ax.legend(loc='lower right', fontsize='small')
    
    # When plotting multiple datasets in a subplots
    if label_xy is True:
        rc = ax.set_ylabel('True Positive Rate')
        rc = ax.set_xlabel('False Positive Rate')
    # Set the master title    
    rc = ax.set_title('ROC curves (dataset={})'.format(training_set))
    return rc

In [37]:
figures_dir = RF_dir / Path("figures")
figures_dir.mkdir(exist_ok=True)

Figure 1a.

In [40]:
fig1_fp = figures_dir / Path("Figure_1a.svg")

fig1, ax = plt.subplots(figsize=(5.32, 5.11))

plot_dataset_roc_curve(best_model, metrics_df, plot_all=True)

fig1.savefig(fig1_fp, dpi=600)

* Figure S1

In [41]:
fig_s1_fp = figures_dir / Path("Figure_S1.svg")
fig, axs = plt.subplots(4,3, figsize=(18, 20))

for i, ts in enumerate(negative_sets):
    plot_dataset_roc_curve(ts, metrics_df, plot_all=True, ax = axs.flat[i], label_xy=False)
# https://napsterinblue.github.io/notes/python/viz/subplots/
else:
    [ax.set_visible(False) for ax in axs.flatten()[i+1:]]
    
# Manually placing axes labels with trial and error
fig.text(0.5, 0.09, 
         'False Positive Rate', 
         ha='center', 
         va='center', 
         fontsize="x-large")
fig.text(0.09, 0.5, 
         'True Positive Rate', 
         ha='center', 
         va='center', 
         rotation='vertical', 
         fontsize="x-large")

# Super title
fig.suptitle("ROC Curves for all datasets", 
             x=0.5, 
             y=0.92, 
             fontsize = "x-large")
# Save it
fig.savefig(fig_s1_fp, dpi=600, bbox_inches='tight')

* Figure S2

In [42]:
fig_s2_fp = figures_dir / Path("Figure_S2.svg")

boxplot_metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc_score',]
fig, axs = plt.subplots(len(boxplot_metrics), 1,
                        figsize=(12, len(boxplot_metrics)*6),
                        )
for i, metric in enumerate(boxplot_metrics):
    title_string = ' '.join(metric.split('_'))
    
    axs.flat[i].set_title(title_string.title(), fontsize="x-large", )
    
    a = sns.boxplot(data=metrics_df, 
                    x='TS', 
                    y=metric, 
                    palette=color_dict, 
                    ax = axs.flat[i]
                   )
    
    a.set_xticklabels([lbl.get_text() for lbl in a.get_xticklabels()],
#                       rotation=60,
#                       horizontalalignment='right',
                      fontsize="large"
                     )

    a.set_ylim([0.4, 1.])
        
    a.set_yticklabels(np.around(a.get_yticks(), 1),
                     fontsize="large")
    a.set_xlabel('')
    a.set_ylabel('')
    
fig.savefig(fig_s2_fp, dpi=600, bbox_inches='tight')
# Gives a 
# .../python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: 
# FixedFormatter should only be used together with FixedLocator

* Figure S3

In [45]:
fig_s3_fp = figures_dir / Path("Figure_S3.svg")
fig, ax = plt.subplots(figsize=(10, 8))
b = sns.barplot(data = metrics_df, 
                x='TS', 
                y='accuracy_abs', 
                ci="sd", 
                palette=color_dict,
                capsize=0.1,
                ax = ax
               )
b.set_xticklabels([lbl.get_text()for lbl in b.get_xticklabels()],
                 size='x-large'
                 )
b.set_ylim([0., 65])
        
b.set_yticklabels(np.around(b.get_yticks(), 0),
                  fontsize=12
                 )
b.set_xlabel('Dataset', fontsize='x-large')
b.set_ylabel('Absolute no. of correct predictions', fontsize='x-large')
b.set_title('Absolute accuracy', 
            fontsize="x-large"
           )
b.axhline(y=62, 
          linestyle='--', 
          color='r'
         )

fig.savefig(fig_s3_fp, dpi=600)

## Features

In [46]:
# Calculate basic stats for all features across all datasets
features_stats = {}
for i, negative in enumerate(negatives_fp_list):
    dataset_name = negative.name.split('.')[0]
    
    set_df = features_df.loc[features_df["TS"] == dataset_name, FEATURES].describe().T
    features_stats[dataset_name] = {'jaccard_score_mean' : set_df.loc['jaccard_score', 'mean'],
                                 'jaccard_score_std' : set_df.loc['jaccard_score', 'std'],
                                'jaccard_score_min' : set_df.loc['jaccard_score', 'min'],
                                'jaccard_score_max' : set_df.loc['jaccard_score', 'max'],
                                 'same_score_mean' : set_df.loc['same_score', 'mean'],
                                 'same_score_std' : set_df.loc['same_score', 'std'],
                                'same_score_min' : set_df.loc['same_score', 'min'],
                                'same_score_max' : set_df.loc['same_score', 'max'],
                                 'inwards_score_mean' : set_df.loc['inwards_score', 'mean'],
                                 'inwards_score_std' : set_df.loc['inwards_score', 'std'],
                                'inwards_score_min' : set_df.loc['inwards_score', 'min'],
                                'inwards_score_max' : set_df.loc['inwards_score', 'max'],
                                 'outwards_score_mean': set_df.loc['outwards_score', 'mean'],
                                 'outwards_score_std': set_df.loc['outwards_score', 'std'],
                                'outwards_score_min' : set_df.loc['outwards_score', 'min'],
                                'outwards_score_max' : set_df.loc['outwards_score', 'max'],
                                'avg_distance_mean': set_df.loc['avg_distance', 'mean'],
                                 'avg_distance_std': set_df.loc['avg_distance', 'std'],
                                'avg_distance_min' : set_df.loc['avg_distance', 'min'],
                                'avg_distance_max' : set_df.loc['avg_distance', 'max'],
                                'mean_ani_mean' : set_df.loc['mean_ani', 'mean'],
                                'mean_ani_std' : set_df.loc['mean_ani', 'std'],
                                'mean_ani_min' : set_df.loc['mean_ani', 'min'],
                                'mean_ani_max' : set_df.loc['mean_ani', 'max'],
                                'mean_aai_mean' : set_df.loc['mean_aai', 'mean'],
                                'mean_aai_std' : set_df.loc['mean_aai', 'std'],
                                'mean_aai_min' : set_df.loc['mean_aai', 'min'],
                                'mean_aai_max' : set_df.loc['mean_aai', 'max'],    
                               }
    
features_stats_df = pd.DataFrame.from_dict(features_stats, orient='index')

# Store them
features_stats_fp = RF_dir / Path("features_stats.tsv")
features_stats_df.T.to_csv(features_stats_fp, sep='\t')


In [47]:
# features_df is calculated on the fly during the 
# randomized search. Go see there.
# TS is for Training Set, VS for Validation Set
feat_imp = features_df.loc[features_df['TS'] == best_model, FEATURES]

fig, ax = plt.subplots(figsize=(5.32,5.11))

features_color_palette = sns.color_palette("colorblind", len(FEATURES))

features_color_dict = dict(zip(FEATURES, features_color_palette))

# fimp : For feature importances
fimp = sns.barplot(data=feat_imp,
                   orient='h',
                   ci="sd",
                   # Order determined with trial and error
                   # TO DO
                   # Sort the df
                   order=[
                       'jaccard_score',
                       'avg_distance',
                       'mean_aai',
                       'mean_ani',
                       'inwards_score',
                       'same_score',
                       'outwards_score'
                   ],
                   palette = features_color_dict,
                   capsize=0.1,
                  )
# Trial and error
fimp.set_xlim([0., 0.45])
        
fimp.set_yticklabels([FEATURES_MAP[lbl.get_text()] for lbl in fimp.get_yticklabels()],
                      fontsize='medium')
fimp.set_xlabel('Gini Importance', fontsize='large')
fimp.set_ylabel('')
fimp.set_title('Feature Importances', fontsize="x-large")

fig.tight_layout()

# Save the fig
feat_importances_fp = figures_dir / Path("Figure_1b.svg")
fig.savefig(feat_importances_fp, dpi= 600)